# Document Parsing with Unstructured for RAG Systems

This comprehensive notebook demonstrates how to use **Unstructured** (v0.18.15) for document parsing in Retrieval-Augmented Generation (RAG) systems. Unstructured is a powerful open-source library that provides tools for ingesting and pre-processing documents for use with Large Language Models.

## What You'll Learn

1. **Core Concepts**: Understanding partition functions, elements, and metadata
2. **Partitioning Strategies**: AUTO, FAST, HI_RES, and OCR_ONLY approaches
3. **File Format Support**: Working with PDFs, Office documents, HTML, Markdown, Images, and more
4. **LangChain Integration**: Seamless integration with LangChain for RAG pipelines
5. **Complete RAG Example**: End-to-end implementation with vector stores

## Key Features of Unstructured

- **28+ Supported File Formats**: PDF, DOCX, XLSX, PPTX, HTML, Markdown, Images, Email, and more
- **Multiple Partitioning Strategies**: Choose the right balance between speed and accuracy
- **Rich Element Types**: Title, NarrativeText, Table, Image, ListItem, etc.
- **Comprehensive Metadata**: Page numbers, coordinates, file types, and custom fields
- **OCR Support**: Built-in OCR for scanned documents and images
- **LangChain Integration**: Native support through `langchain-unstructured`

---
## 1. Installation & Setup

First, let's install all the required packages. Unstructured supports various "extras" for different file formats.

In [1]:
# Install Unstructured with all common extras
# The extras enable support for specific file formats

# Core installation with common document format support
#!uv pip install -q "unstructured==0.18.15"

# Install extras for specific file formats
# - pdf: PDF document support
# - docx: Microsoft Word documents
# - xlsx: Microsoft Excel spreadsheets
# - pptx: Microsoft PowerPoint presentations
# - html: HTML web pages
# - md: Markdown files
# - image: Image files (PNG, JPEG, etc.)
# - email: Email files (EML, MSG)
#!uv pip install -q "unstructured[pdf,docx,xlsx,pptx,html,md,image,email]"

# Install LangChain integration packages
#!uv pip install -q langchain-unstructured langchain-community langchain-openai

# Install additional dependencies
#!uv pip install -q chromadb python-dotenv

#print("Installation complete!")

In [ ]:
# Verify installation and check version
import unstructured
from importlib.metadata import version

print(version("unstructured"))

0.18.21


In [ ]:
# Load environment variables for API keys
# Create a .env file with your OPENAI_API_KEY

import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify OpenAI API key is set (for RAG examples later)
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API key loaded successfully")
else:
    print("Warning: OPENAI_API_KEY not found. RAG examples will not work.")
    print("Create a .env file with: OPENAI_API_KEY=your-key-here")

OpenAI API key loaded successfully


In [2]:
# Import commonly used modules
# These will be used throughout the notebook

from unstructured.partition.auto import partition
from unstructured.partition.pdf import partition_pdf
from unstructured.partition.html import partition_html
from unstructured.partition.md import partition_md
from unstructured.partition.docx import partition_docx
from unstructured.partition.xlsx import partition_xlsx
from unstructured.partition.pptx import partition_pptx
from unstructured.partition.image import partition_image

# Chunking utilities
from unstructured.chunking.title import chunk_by_title
from unstructured.chunking.basic import chunk_elements

# Element types for type checking
from unstructured.documents.elements import (
    Title,
    NarrativeText,
    Table,
    ListItem,
    Image,
    Header,
    Footer,
    Text,
    ElementMetadata,
)

import json
from pathlib import Path

print("All imports successful!")

c:\learning_notes\RAGG\simple-rag-langchain-1\Data Parsers\document-parsers-rag\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports successful!


---
## 2. Core Concepts

### Understanding the `partition()` Function

The `partition()` function is the universal entry point for document processing in Unstructured. It automatically detects the file type and applies the appropriate partitioning logic.

### Element Types

Unstructured extracts structured elements from documents:

| Element Type | Description |
|-------------|-------------|
| `Title` | Headings and section titles |
| `NarrativeText` | Paragraphs and body text |
| `Table` | Tabular data |
| `ListItem` | List elements (bulleted, numbered) |
| `Image` | Embedded images |
| `Header` | Page headers |
| `Footer` | Page footers |
| `Text` | Generic text elements |
|`Formula` | An element containing formulas in a document.|
|`FigureCaption` | An element for capturing text associated with figure captions.|
|`Address` | A text element for capturing physical addresses.|
|`EmailAddress` | A text element for capturing email addresses.|
|`PageBreak` | An element for capturing page breaks.|
| `CodeSnippet` | An element for capturing code snippets.|
|`PageNumber` | An element for capturing page numbers.|
|`UncategorizedText` | Base element for capturing free text from within document.|

In [ ]:
# Basic example: Partition a local HTML file
# Using the sample HTML file from the sample_documents folder

html_path = "sample_documents_multi/sample.html"

# The partition() function automatically detects the file type
# and applies the appropriate parsing logic
elements = partition(filename=html_path)

# Display the number of elements extracted
print(f"Total elements extracted: {type(elements)}")
print("\n" + "="*60 + "\n")

# Show each element with its type
for i, element in enumerate(elements):
    element_type = type(element).__name__
    # Truncate long text for display
    text_preview = element.text[:100] + "..." if len(element.text) > 100 else element.text
    print(f"[{i}] {element_type}: {text_preview}")
    print("-" * 60)

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Total elements extracted: <class 'list'>


[0] Title: 1. Overview
------------------------------------------------------------
[1] NarrativeText: Document parsing is the process of analyzing and extracting structured information from various docu...
------------------------------------------------------------
[2] Title: 1.1 Key Benefits
------------------------------------------------------------
[3] ListItem: Automated data extraction
------------------------------------------------------------
[4] ListItem: Structured content analysis
------------------------------------------------------------
[5] ListItem: Integration with AI/ML pipelines
------------------------------------------------------------
[6] ListItem: Support for multiple formats
------------------------------------------------------------
[7] Title: 2. Core Features
------------------------------------------------------------
[8] NarrativeText: Modern document parsers offer a variety of features:
-----------------------

: 

In [5]:
# Understanding ElementMetadata
# Each element has rich metadata attached to it

# Get the first Title element
title_elements = [e for e in elements if isinstance(e, Title)]

if title_elements:
    first_title = title_elements[0]
    
    print("Element Text:")
    print(first_title.text)
    print("\n" + "="*60 + "\n")
    
    print("Element Metadata:")
    metadata = first_title.metadata
    
    # Display available metadata fields
    metadata_dict = metadata.to_dict()
    for key, value in metadata_dict.items():
        if value is not None:
            print(f"  {key}: {value}")

Element Text:
1. Overview


Element Metadata:
  category_depth: 1
  last_modified: 2025-11-28T09:19:12
  languages: ['eng']
  file_directory: sample_documents
  filename: sample.html
  filetype: text/html


In [6]:
# Filter elements by type
# This is useful when you need specific content from a document

def analyze_elements(elements):
    """Analyze and categorize extracted elements."""
    
    # Count elements by type
    type_counts = {}
    for element in elements:
        element_type = type(element).__name__
        type_counts[element_type] = type_counts.get(element_type, 0) + 1
    
    print("Element Distribution:")
    print("-" * 30)
    for elem_type, count in sorted(type_counts.items()):
        print(f"  {elem_type}: {count}")
    
    return type_counts

# Analyze the HTML elements
type_counts = analyze_elements(elements)

Element Distribution:
------------------------------
  ListItem: 8
  NarrativeText: 6
  Table: 1
  Title: 5


In [7]:
# Extract specific element types
# Example: Get all narrative text for summarization

narrative_texts = [e for e in elements if isinstance(e, NarrativeText)]

print(f"Found {len(narrative_texts)} NarrativeText elements:\n")
for i, text_elem in enumerate(narrative_texts[:5]):  # Show first 5
    print(f"[{i+1}] {text_elem.text}")
    print()

Found 6 NarrativeText elements:

[1] Document parsing is the process of analyzing and extracting structured information from various document formats. This includes PDFs, Word documents, HTML pages, and more.

[2] Modern document parsers offer a variety of features:

[3] Retrieval-Augmented Generation (RAG) systems benefit significantly from proper document parsing:

[4] "Effective document parsing is the foundation of any successful RAG implementation."

[5] Here's a simple example of using a document parser:



In [8]:
# Extract tables from the document
# Tables maintain their structure and can be exported to different formats

table_elements = [e for e in elements if isinstance(e, Table)]

print(f"Found {len(table_elements)} Table elements:\n")

for i, table in enumerate(table_elements):
    print(f"Table {i+1}:")
    print(table.text)
    print()
    
    # If table has HTML representation in metadata
    if hasattr(table.metadata, 'text_as_html') and table.metadata.text_as_html:
        print("HTML representation available in metadata.text_as_html")
    print("-" * 60)

Found 1 Table elements:

Table 1:
Feature Description Use Case OCR Support Optical Character Recognition for scanned documents Scanned PDFs, Images Table Extraction Structured table data extraction Financial reports, Data tables Layout Analysis Understanding document structure Academic papers, Legal documents Image Processing Extract and classify images Technical manuals, Presentations

HTML representation available in metadata.text_as_html
------------------------------------------------------------


---
## 3. Partitioning Strategies

Unstructured offers different partitioning strategies to balance speed and accuracy:

| Strategy | Speed | Accuracy | Use Case |
|----------|-------|----------|----------|
| `auto` | Variable | Good | Default choice, selects best strategy |
| `fast` | Fast | Basic | Quick processing, no models |
| `hi_res` | Slow | Best | Complex layouts, OCR, tables |
| `ocr_only` | Medium | For scanned | Purely OCR-based extraction |

In [9]:
# Download a sample PDF for demonstrating different strategies
# Using the Docling paper from arXiv as an example

import urllib.request
import os

# Create sample_documents directory if it doesn't exist
os.makedirs("sample_documents", exist_ok=True)

pdf_url = "https://arxiv.org/pdf/2408.09869"
pdf_path = "sample_documents/docling_paper.pdf"

# Download only if not already present
if not os.path.exists(pdf_path):
    print(f"Downloading PDF from {pdf_url}...")
    urllib.request.urlretrieve(pdf_url, pdf_path)
    print(f"Downloaded to {pdf_path}")
else:
    print(f"PDF already exists at {pdf_path}")

PDF already exists at sample_documents/docling_paper.pdf


In [10]:
# Strategy 1: AUTO (Default)
# Automatically selects the best strategy based on the document

import time

print("Strategy: AUTO (default)")
print("="*60)

start_time = time.time()

elements_auto = partition(
    filename=pdf_path,
    strategy="auto",  # This is the default
)

elapsed_time = time.time() - start_time

print(f"Time taken: {elapsed_time:.2f} seconds")
print(f"Elements extracted: {len(elements_auto)}")

# Show element distribution
analyze_elements(elements_auto)

Strategy: AUTO (default)
Time taken: 3.94 seconds
Elements extracted: 276
Element Distribution:
------------------------------
  Footer: 10
  ListItem: 4
  NarrativeText: 75
  Text: 86
  Title: 101


{'Text': 86, 'Title': 101, 'NarrativeText': 75, 'Footer': 10, 'ListItem': 4}

In [11]:
# Strategy 2: FAST
# Quick processing without using ML models
# Good for simple documents or when speed is critical

print("Strategy: FAST")
print("="*60)

start_time = time.time()

elements_fast = partition(
    filename=pdf_path,
    strategy="fast",  # No ML models, faster processing
)

elapsed_time = time.time() - start_time

print(f"Time taken: {elapsed_time:.2f} seconds")
print(f"Elements extracted: {len(elements_fast)}")

# Show element distribution
analyze_elements(elements_fast)

Strategy: FAST
Time taken: 0.85 seconds
Elements extracted: 276
Element Distribution:
------------------------------
  Footer: 10
  ListItem: 4
  NarrativeText: 75
  Text: 86
  Title: 101


{'Text': 86, 'Title': 101, 'NarrativeText': 75, 'Footer': 10, 'ListItem': 4}

In [12]:
# Strategy 3: HI_RES
# High-resolution processing with ML models
# Best for complex documents with tables, figures, and mixed layouts
# Note: This requires additional dependencies and takes longer

print("Strategy: HI_RES")
print("="*60)
print("Note: hi_res strategy uses ML models and may take longer")
print()

start_time = time.time()

try:
    elements_hires = partition_pdf(
        filename=pdf_path,
        strategy="hi_res",
        infer_table_structure=True,  # Extract table structure
    )
    
    elapsed_time = time.time() - start_time
    
    print(f"Time taken: {elapsed_time:.2f} seconds")
    print(f"Elements extracted: {len(elements_hires)}")
    
    # Show element distribution
    analyze_elements(elements_hires)
    
except Exception as e:
    print(f"hi_res strategy requires additional dependencies.")
    print(f"Error: {e}")
    print("\nTo enable hi_res, install: pip install 'unstructured[pdf-infer]'")

Strategy: HI_RES
Note: hi_res strategy uses ML models and may take longer



The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


Time taken: 63.62 seconds
Elements extracted: 327
Element Distribution:
------------------------------
  FigureCaption: 2
  Footer: 2
  Header: 4
  Image: 20
  ListItem: 20
  NarrativeText: 58
  Table: 4
  Text: 192
  Title: 25


In [13]:
# Compare extracted text between strategies
# Let's look at the first few elements from each strategy

def show_first_elements(elements, n=5, strategy_name=""):
    """Display the first n elements from a partition result."""
    print(f"\n{'='*60}")
    print(f"First {n} elements from {strategy_name} strategy:")
    print("="*60)
    
    for i, elem in enumerate(elements[:n]):
        elem_type = type(elem).__name__
        text = elem.text[:150] + "..." if len(elem.text) > 150 else elem.text
        print(f"\n[{i}] {elem_type}:")
        print(f"    {text}")

# Compare AUTO and FAST strategies
show_first_elements(elements_auto, n=5, strategy_name="AUTO")
show_first_elements(elements_fast, n=5, strategy_name="FAST")


First 5 elements from AUTO strategy:

[0] Text:
    4 2 0 2 c e D 9

[1] Title:
    ] L C . s c [

[2] Text:
    5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a

[3] Title:
    Docling Technical Report

[4] Title:
    Version 1.0

First 5 elements from FAST strategy:

[0] Text:
    4 2 0 2 c e D 9

[1] Title:
    ] L C . s c [

[2] Text:
    5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a

[3] Title:
    Docling Technical Report

[4] Title:
    Version 1.0


---
## 4. Supported File Formats

Unstructured supports 28+ file formats. Let's explore examples for the most common ones.

### 4.1 PDF Documents

In [5]:
# PDF Partitioning with various options
# Using partition_pdf for more control over PDF-specific settings

from unstructured.partition.pdf import partition_pdf

pdf_path = "sample_documents_multi/docling_paper.pdf"
# Basic PDF partitioning
pdf_elements = partition_pdf(
    filename=pdf_path,
    strategy="fast",  # Use fast for this demo
)

print(f"PDF Elements: {len(pdf_elements)}")
print("\nFirst 3 elements:")
for elem in pdf_elements[:3]:
    print(f"  - {type(elem).__name__}: {elem.text[:80]}...")

PDF Elements: 276

First 3 elements:
  - Text: 4 2 0 2 c e D 9...
  - Title: ] L C . s c [...
  - Text: 5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a...


In [ ]:
for e in pdf_elements:
    mtdata = e.metadata.to_dict()
    mtdata["source"]="pdf"
    print(mtdata)

{'coordinates': {'points': ((16.34, 216.16000000000008), (16.34, 308.36), (36.34, 308.36), (36.34, 216.16000000000008)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': 'sample_documents_multi', 'filename': 'docling_paper.pdf', 'last_modified': '2026-01-19T01:52:10', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/pdf', 'source': 'pdf'}
{'coordinates': {'points': ((16.34, 318.35999999999996), (16.34, 378.9), (36.34, 378.9), (36.34, 318.35999999999996)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': 'sample_documents_multi', 'filename': 'docling_paper.pdf', 'last_modified': '2026-01-19T01:52:10', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/pdf', 'source': 'pdf'}
{'coordinates': {'points': ((16.34, 388.9), (16.34, 555.0), (36.34, 555.0), (36.34, 388.9)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': 'sample_documents_multi

: 

In [15]:
# PDF with table extraction enabled
# This extracts structured table data from PDFs

pdf_elements_tables = partition_pdf(
    filename=pdf_path,
    strategy="fast",
    include_page_breaks=True,  # Mark page boundaries
)

# Find tables in the document
tables = [e for e in pdf_elements_tables if isinstance(e, Table)]
print(f"Found {len(tables)} tables in the PDF")

# Show first table if found
if tables:
    print("\nFirst table content:")
    print(tables[0].text[:500])

Found 0 tables in the PDF


In [16]:
# Get elements with page numbers
# Page metadata is useful for citations and navigation

# Group elements by page
pages = {}
for elem in pdf_elements_tables:
    page_num = elem.metadata.page_number if elem.metadata.page_number else 0
    if page_num not in pages:
        pages[page_num] = []
    pages[page_num].append(elem)

print(f"Document has {len(pages)} pages")
for page_num in sorted(pages.keys())[:3]:  # Show first 3 pages
    print(f"\nPage {page_num}: {len(pages[page_num])} elements")

Document has 10 pages

Page 0: 9 elements

Page 1: 15 elements

Page 2: 19 elements


### 4.2 HTML Documents

In [17]:
# HTML Partitioning
# Unstructured can parse both local HTML files and URLs

from unstructured.partition.html import partition_html

# Partition the sample HTML file
html_elements = partition_html(filename="sample_documents/sample.html")

print(f"HTML Elements: {len(html_elements)}")
analyze_elements(html_elements)

HTML Elements: 20
Element Distribution:
------------------------------
  ListItem: 8
  NarrativeText: 6
  Table: 1
  Title: 5


{'Title': 5, 'NarrativeText': 6, 'ListItem': 8, 'Table': 1}

In [ ]:
# HTML from URL (web scraping)
# You can also partition HTML directly from a   URL

try:
    url_elements = partition_html(
        url="https://docling-project.github.io/docling/",
    )
    
    print(f"Elements from URL: {len(url_elements)}")
    for elem in url_elements[:5]:
        print(f"  - {type(elem).__name__}: {elem.text[:60]}...")
except Exception as e:
    print(f"Could not fetch URL: {e}")

Elements from URL: 54
  - Title: Documentation...
  - Image: Docling...
  - Image: DS4SD%2Fdocling | Trendshift...
  - Image: arXiv...
  - Image: PyPI version...


In [19]:
# HTML from string content
# Useful when you have HTML content in memory

html_content = """
<html>
<body>
    <h1>Sample Document</h1>
    <p>This is a paragraph with some <strong>bold text</strong>.</p>
    <ul>
        <li>Item 1</li>
        <li>Item 2</li>
        <li>Item 3</li>
    </ul>
    <table>
        <tr><th>Name</th><th>Value</th></tr>
        <tr><td>A</td><td>100</td></tr>
        <tr><td>B</td><td>200</td></tr>
    </table>
</body>
</html>
"""

elements_from_string = partition_html(text=html_content)

print(f"Elements from string: {len(elements_from_string)}")
for elem in elements_from_string:
    print(f"  - {type(elem).__name__}: {elem.text}")

Elements from string: 6
  - Title: Sample Document
  - NarrativeText: This is a paragraph with some bold text.
  - ListItem: Item 1
  - ListItem: Item 2
  - ListItem: Item 3
  - Table: Name Value A 100 B 200


### 4.3 Markdown Documents

In [20]:
# Markdown Partitioning
# Parses markdown syntax and extracts structured elements

from unstructured.partition.md import partition_md

# Partition the sample markdown file
md_elements = partition_md(filename="sample_documents/sample.md")

print(f"Markdown Elements: {len(md_elements)}")
analyze_elements(md_elements)

Markdown Elements: 44
Element Distribution:
------------------------------
  ListItem: 15
  NarrativeText: 12
  Table: 2
  Text: 1
  Title: 14


{'Title': 14, 'NarrativeText': 12, 'ListItem': 15, 'Table': 2, 'Text': 1}

In [21]:
# Show markdown structure
# Headings become Title elements, paragraphs become NarrativeText

print("Markdown Structure:")
print("="*60)

for elem in md_elements[:15]:
    elem_type = type(elem).__name__
    text = elem.text[:80] + "..." if len(elem.text) > 80 else elem.text
    print(f"{elem_type:15} | {text}")

Markdown Structure:
Title           | Document Parsing Best Practices
NarrativeText   | A comprehensive guide to document parsing for RAG systems.
Title           | Table of Contents
ListItem        | Introduction
ListItem        | Supported Formats
ListItem        | Parsing Strategies
ListItem        | Integration Guide
Title           | Introduction
NarrativeText   | Document parsing is a critical component in modern AI applications. It enables t...
ListItem        | Build searchable knowledge bases
ListItem        | Create training datasets for machine learning
ListItem        | Enable semantic search and retrieval
ListItem        | Power question-answering systems
NarrativeText   | Note: The quality of document parsing directly impacts the performance of downst...
Title           | Supported Formats


In [23]:
# Markdown from string
# Useful for processing markdown content in memory

md_string = """
# Main Title

This is an introduction paragraph.

## Section 1

Here's some content with:
- Bullet point 1
- Bullet point 2

## Section 2

```python
def hello():
    print("Hello, World!")
```

| Column A | Column B |
|----------|----------|
| Value 1  | Value 2  |
"""

md_string_elements = partition_md(text=md_string)

print(f"Elements from markdown string: {len(md_string_elements)}")
for elem in md_string_elements:
    print(f"  - {type(elem).__name__}: {elem.text[:60]}..." if len(elem.text) > 60 else f"  - {type(elem).__name__}: {elem.text}")

Elements from markdown string: 7
  - Title: Main Title
  - NarrativeText: This is an introduction paragraph.
  - Title: Section 1
  - NarrativeText: Here's some content with: - Bullet point 1 - Bullet point 2
  - Title: Section 2
  - Text: def hello():
    print("Hello, World!")
  - Table: Column A Column B Value 1 Value 2


### 4.4 Microsoft Office Documents

In [24]:
# Create a sample DOCX file for demonstration
# This requires python-docx library

try:
    from docx import Document as DocxDocument
    from docx.shared import Inches
    
    # Create a new document
    doc = DocxDocument()
    
    # Add content
    doc.add_heading('Sample Word Document', 0)
    doc.add_paragraph('This is a sample Word document created for testing Unstructured.')
    
    doc.add_heading('Section 1: Introduction', level=1)
    doc.add_paragraph('Unstructured is a powerful library for document parsing.')
    
    doc.add_heading('Section 2: Features', level=1)
    doc.add_paragraph('Key features include:')
    
    # Add a bulleted list
    doc.add_paragraph('Multiple file format support', style='List Bullet')
    doc.add_paragraph('OCR capabilities', style='List Bullet')
    doc.add_paragraph('Table extraction', style='List Bullet')
    
    # Add a simple table
    table = doc.add_table(rows=3, cols=2)
    table.style = 'Table Grid'
    cells = table.rows[0].cells
    cells[0].text = 'Feature'
    cells[1].text = 'Status'
    cells = table.rows[1].cells
    cells[0].text = 'PDF Support'
    cells[1].text = 'Available'
    cells = table.rows[2].cells
    cells[0].text = 'OCR'
    cells[1].text = 'Available'
    
    # Save the document
    docx_path = 'sample_documents/sample1.docx'
    doc.save(docx_path)
    print(f"Created sample DOCX at: {docx_path}")
    
except ImportError:
    print("python-docx not installed. Install with: pip install python-docx")
    docx_path = None

Created sample DOCX at: sample_documents/sample1.docx


In [24]:
# DOCX Partitioning
# Parse Microsoft Word documents

from unstructured.partition.docx import partition_docx

if docx_path and os.path.exists(docx_path):
    docx_elements = partition_docx(filename=docx_path)
    
    print(f"DOCX Elements: {len(docx_elements)}")
    analyze_elements(docx_elements)
    
    print("\nContent:")
    for elem in docx_elements:
        print(f"  - {type(elem).__name__}: {elem.text}")
else:
    print("No DOCX file available for demo")

DOCX Elements: 10
Element Distribution:
------------------------------
  ListItem: 3
  NarrativeText: 3
  Table: 1
  Title: 3

Content:
  - Title: Sample Word Document
  - NarrativeText: This is a sample Word document created for testing Unstructured.
  - Title: Section 1: Introduction
  - NarrativeText: Unstructured is a powerful library for document parsing.
  - Title: Section 2: Features
  - NarrativeText: Key features include:
  - ListItem: Multiple file format support
  - ListItem: OCR capabilities
  - ListItem: Table extraction
  - Table: Feature Status PDF Support Available OCR Available


In [25]:
# Create a sample XLSX file for demonstration

try:
    import openpyxl
    
    # Create a new workbook
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Data"
    
    # Add headers
    ws['A1'] = 'Product'
    ws['B1'] = 'Category'
    ws['C1'] = 'Price'
    ws['D1'] = 'Quantity'
    
    # Add data
    data = [
        ('Laptop', 'Electronics', 999.99, 50),
        ('Phone', 'Electronics', 599.99, 100),
        ('Desk', 'Furniture', 299.99, 25),
        ('Chair', 'Furniture', 149.99, 75),
        ('Monitor', 'Electronics', 349.99, 40),
    ]
    
    for row_idx, row_data in enumerate(data, start=2):
        for col_idx, value in enumerate(row_data, start=1):
            ws.cell(row=row_idx, column=col_idx, value=value)
    
    # Save the workbook
    xlsx_path = 'sample_documents/sample1.xlsx'
    wb.save(xlsx_path)
    print(f"Created sample XLSX at: {xlsx_path}")
    
except ImportError:
    print("openpyxl not installed. Install with: pip install openpyxl")
    xlsx_path = None

Created sample XLSX at: sample_documents/sample1.xlsx


In [29]:
# XLSX Partitioning
# Parse Microsoft Excel spreadsheets

from unstructured.partition.xlsx import partition_xlsx

xlsx_path = "sample_documents/sample.xlsx"
if xlsx_path and os.path.exists(xlsx_path):
    xlsx_elements = partition_xlsx(filename=xlsx_path)
    
    print(f"XLSX Elements: {len(xlsx_elements)}")
    analyze_elements(xlsx_elements)
    
    print("\nContent:")
    for elem in xlsx_elements:
        text = elem.text[:200] + "..." if len(elem.text) > 200 else elem.text
        print(f"  - {type(elem).__name__}: {text}")
else:
    print("No XLSX file available for demo")

XLSX Elements: 11
Element Distribution:
------------------------------
  NarrativeText: 4
  Table: 3
  Text: 2
  Title: 2

Content:
  - Table: Dates Modules 2 November 2025 9 November 2025 16 November 2025 23 November 2025 30 November 2025 7 December 2025 Module 4: Document Parsers + Module 5 (Part 1): LlamaIndex Fundamentals 14 December 202...
  - Text: 12 April 2026
  - Title: Black Friday 2025: AI Services & Tools Discount Guide
  - Table: by Seulki Kang https://www.linkedin.com/in/seulki-kang/
  - Title: Crawled by Genspark AI Sheet
  - NarrativeText: Want to try Genspark yourself?
  - NarrativeText: 🔗 Get 1,000 free credits: https://www.genspark.ai/invite_member?invite_code=NTQxMmI5M2ZMOTRhMUwyODg2TDhjZmZMNzE3ODRhYmI0Y2Jl
  - NarrativeText: 🔗 10% discount code: https://www.genspark.ai/?via=seulki875 (US) / https://www.genspark.ai/?via=seulki (Other Countries)
  - Table: Name Description URL Benefits Monthly Plan Annual Plan Notes ⭐ Popular AI Services (Discount Info) Perplexity Pr

In [27]:
# Create a sample PPTX file for demonstration

try:
    from pptx import Presentation
    from pptx.util import Inches, Pt
    
    # Create a presentation
    prs = Presentation()
    
    # Add title slide
    title_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(title_slide_layout)
    title = slide.shapes.title
    subtitle = slide.placeholders[1]
    title.text = "Unstructured Demo"
    subtitle.text = "Document Parsing for RAG"
    
    # Add content slide
    bullet_slide_layout = prs.slide_layouts[1]
    slide = prs.slides.add_slide(bullet_slide_layout)
    shapes = slide.shapes
    title_shape = shapes.title
    body_shape = shapes.placeholders[1]
    title_shape.text = "Key Features"
    tf = body_shape.text_frame
    tf.text = "Multiple file formats supported"
    p = tf.add_paragraph()
    p.text = "OCR for scanned documents"
    p.level = 0
    p = tf.add_paragraph()
    p.text = "Table extraction"
    p.level = 0
    
    # Save the presentation
    pptx_path = 'sample_documents/sample1.pptx'
    prs.save(pptx_path)
    print(f"Created sample PPTX at: {pptx_path}")
    
except ImportError:
    print("python-pptx not installed. Install with: pip install python-pptx")
    pptx_path = None

Created sample PPTX at: sample_documents/sample1.pptx


In [30]:
# PPTX Partitioning
# Parse Microsoft PowerPoint presentations

from unstructured.partition.pptx import partition_pptx

pptx_path = "sample_documents/sample.pptx"
if pptx_path and os.path.exists(pptx_path):
    pptx_elements = partition_pptx(filename=pptx_path)
    
    print(f"PPTX Elements: {len(pptx_elements)}")
    analyze_elements(pptx_elements)
    
    print("\nContent:")
    for elem in pptx_elements:
        print(f"  - {type(elem).__name__}: {elem.text}")
else:
    print("No PPTX file available for demo")

PPTX Elements: 23
Element Distribution:
------------------------------
  NarrativeText: 2
  PageBreak: 14
  Text: 3
  Title: 4

Content:
  - Title: Stop searching. Start building
  - Title: https://docrag.evolvue.ai/
  - PageBreak: 
  - PageBreak: 
  - PageBreak: 
  - PageBreak: 
  - PageBreak: 
  - PageBreak: 
  - Title: Universal Compatibility
  - NarrativeText: Seamlessly integrate with the tools you already use and love (with more to come soon!)
  - PageBreak: 
  - Title: Multi-Model Intelligence
  - NarrativeText: Choose from the best AI models in the industry to get accurate and contextual responses for every query
  - PageBreak: 
  - PageBreak: 
  - PageBreak: 
  - PageBreak: 
  - PageBreak: 
  - Text: 1.28M+
  - Text: 281k+
  - Text: 361k+
  - PageBreak: 
  - PageBreak: 


### 4.5 Image Files (with OCR)

In [29]:
# Create a sample image with text for OCR demo

try:
    from PIL import Image, ImageDraw, ImageFont
    
    # Create a white image
    img = Image.new('RGB', (800, 400), color='white')
    draw = ImageDraw.Draw(img)
    
    # Try to use a basic font, fall back to default if not available
    try:
        font_title = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 36)
        font_text = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf", 24)
    except:
        font_title = ImageFont.load_default()
        font_text = ImageFont.load_default()
    
    # Add text to the image
    draw.text((50, 30), "Document Parsing Demo", fill='black', font=font_title)
    draw.text((50, 100), "This is sample text for OCR testing.", fill='black', font=font_text)
    draw.text((50, 150), "Unstructured can extract text from images.", fill='black', font=font_text)
    draw.text((50, 200), "Features:", fill='black', font=font_text)
    draw.text((70, 250), "• Multiple language support", fill='black', font=font_text)
    draw.text((70, 300), "• Various OCR backends", fill='black', font=font_text)
    draw.text((70, 350), "• High accuracy extraction", fill='black', font=font_text)
    
    # Save the image
    image_path = 'sample_documents/sample_text_image.png'
    img.save(image_path)
    print(f"Created sample image at: {image_path}")
    
except ImportError:
    print("Pillow not installed. Install with: pip install Pillow")
    image_path = None

Created sample image at: sample_documents/sample_text_image.png


In [30]:
# Image Partitioning with OCR
# Extract text from images using OCR

from unstructured.partition.image import partition_image

if image_path and os.path.exists(image_path):
    try:
        # Partition the image using OCR
        image_elements = partition_image(
            filename=image_path,
            strategy="auto",  # Will use OCR automatically
        )
        
        print(f"Image Elements: {len(image_elements)}")
        analyze_elements(image_elements)
        
        print("\nExtracted Text:")
        for elem in image_elements:
            print(f"  - {type(elem).__name__}: {elem.text}")
            
    except Exception as e:
        print(f"OCR error: {e}")
        print("\nOCR requires additional dependencies.")
        print("Install with: pip install 'unstructured[local-inference]'")
        print("Also ensure Tesseract OCR is installed on your system.")
else:
    print("No image file available for demo")

Image Elements: 7
Element Distribution:
------------------------------
  Footer: 1
  Header: 1
  NarrativeText: 5

Extracted Text:
  - Header: Document Parsing Demo
  - NarrativeText: This issample text for OCR testing.
  - NarrativeText: Unstructured can extract text from images.
  - NarrativeText: Features
  - NarrativeText: 8 Multiple language support
  - NarrativeText: B VariousOCR backends
  - Footer: 8 High accuracy extraction


---
## 5. Element Types Deep Dive

Let's explore the different element types and their properties in detail.

In [31]:
# Detailed look at Title elements
# Titles represent headings and section headers

# Get titles from the markdown document
titles = [e for e in md_elements if isinstance(e, Title)]

print("Title Elements Analysis")
print("="*60)
print(f"Total titles found: {len(titles)}")
print()

for i, title in enumerate(titles[:5]):
    print(f"Title {i+1}:")
    print(f"  Text: {title.text}")
    print(f"  ID: {title.id}")
    
    # Check for category depth (heading level)
    if hasattr(title.metadata, 'category_depth'):
        print(f"  Category Depth: {title.metadata.category_depth}")
    print()

Title Elements Analysis
Total titles found: 14

Title 1:
  Text: Document Parsing Best Practices
  ID: 9ea1b6ba00245b0da1c77098a9e130d5
  Category Depth: 0

Title 2:
  Text: Table of Contents
  ID: bf0aeb6089cc5feb43bf6f5734f3d849
  Category Depth: 1

Title 3:
  Text: Introduction
  ID: be39b6913500d3c48b592988ca400449
  Category Depth: 1

Title 4:
  Text: Supported Formats
  ID: bfe92481356969a08531b7d9e04d44ac
  Category Depth: 1

Title 5:
  Text: Primary Formats
  ID: e88f38a1f4f2fab33f199c2dd9224d98
  Category Depth: 2



In [ ]:
# NarrativeText elements - main body content

narrative_texts = [e for e in md_elements if isinstance(e, NarrativeText)]

print("NarrativeText Elements Analysis")
print("="*60)
print(f"Total narrative texts: {len(narrative_texts)}")
print()

# Calculate text statistics
total_chars = sum(len(n.text) for n in narrative_texts)
avg_chars = total_chars / len(narrative_texts) if narrative_texts else 0

print(f"Total characters: {total_chars}")
print(f"Average characters per element: {avg_chars:.1f}")\

    
print()

print("Sample narrative texts:")
for text in narrative_texts[:3]:
    preview = text.text[:100] + "..." if len(text.text) > 100 else text.text
    print(f"  - {preview}")

NarrativeText Elements Analysis
Total narrative texts: 12

Total characters: 1227
Average characters per element: 102.2

Sample narrative texts:
  - A comprehensive guide to document parsing for RAG systems.
  - Document parsing is a critical component in modern AI applications. It enables the extraction of str...
  - Note: The quality of document parsing directly impacts the performance of downstream AI applications...


In [33]:
# Table elements - structured tabular data

# Get tables from HTML (which has a table)
tables = [e for e in html_elements if isinstance(e, Table)]

print("Table Elements Analysis")
print("="*60)
print(f"Total tables: {len(tables)}")
print()

for i, table in enumerate(tables):
    print(f"Table {i+1}:")
    print(f"  Content: {table.text[:300]}..." if len(table.text) > 300 else f"  Content: {table.text}")
    
    # Check for HTML representation
    if hasattr(table.metadata, 'text_as_html') and table.metadata.text_as_html:
        print(f"  HTML available: Yes")
    print()

Table Elements Analysis
Total tables: 1

Table 1:
  Content: Feature Description Use Case OCR Support Optical Character Recognition for scanned documents Scanned PDFs, Images Table Extraction Structured table data extraction Financial reports, Data tables Layout Analysis Understanding document structure Academic papers, Legal documents Image Processing Extrac...
  HTML available: Yes



In [34]:
# ListItem elements - bullet points and numbered items

list_items = [e for e in html_elements if isinstance(e, ListItem)]

print("ListItem Elements Analysis")
print("="*60)
print(f"Total list items: {len(list_items)}")
print()

print("List items found:")
for item in list_items:
    print(f"  • {item.text}")

ListItem Elements Analysis
Total list items: 8

List items found:
  • Automated data extraction
  • Structured content analysis
  • Integration with AI/ML pipelines
  • Support for multiple formats
  • Knowledge Base Creation: Convert documents into searchable chunks
  • Semantic Search: Enable meaning-based document retrieval
  • Question Answering: Provide accurate answers from document context
  • Document Summarization: Generate concise summaries


In [35]:
# Converting elements to different formats

# Convert to dictionary
element_dict = html_elements[0].to_dict()
print("Element as Dictionary:")
print(json.dumps(element_dict, indent=2))

Element as Dictionary:
{
  "type": "Title",
  "element_id": "2a000fb58dc445e313c5dffe0ef3a614",
  "text": "1. Overview",
  "metadata": {
    "category_depth": 1,
    "last_modified": "2025-11-28T09:19:12",
    "languages": [
      "eng"
    ],
    "file_directory": "sample_documents",
    "filename": "sample.html",
    "filetype": "text/html"
  }
}


In [36]:
# Convert all elements to a list of dictionaries
# Useful for serialization and export

from unstructured.staging.base import elements_to_json

# Convert to JSON string
json_output = elements_to_json(html_elements[:5])
print("First 5 elements as JSON:")
print(json_output[:1000] + "..." if len(json_output) > 1000 else json_output)

First 5 elements as JSON:
[
    {
        "element_id": "2a000fb58dc445e313c5dffe0ef3a614",
        "metadata": {
            "category_depth": 1,
            "file_directory": "sample_documents",
            "filename": "sample.html",
            "filetype": "text/html",
            "languages": [
                "eng"
            ],
            "last_modified": "2025-11-28T09:19:12"
        },
        "text": "1. Overview",
        "type": "Title"
    },
    {
        "element_id": "0111c3838e668907531ba2247a8d97b1",
        "metadata": {
            "file_directory": "sample_documents",
            "filename": "sample.html",
            "filetype": "text/html",
            "languages": [
                "eng"
            ],
            "last_modified": "2025-11-28T09:19:12",
            "parent_id": "2a000fb58dc445e313c5dffe0ef3a614"
        },
        "text": "Document parsing is the process of analyzing and extracting structured information from various document formats. This incl

---
## 6. Working with Metadata

Each element in Unstructured contains rich metadata that can be used for filtering, citations, and enhanced retrieval.

In [37]:
# Exploring element metadata

# Get a sample element from PDF
sample_element = pdf_elements[0]

print("Element Metadata Fields")
print("="*60)

# Get all metadata as dictionary
metadata_dict = sample_element.metadata.to_dict()

for key, value in metadata_dict.items():
    if value is not None:
        print(f"{key}: {value}")

Element Metadata Fields
coordinates: {'points': ((16.34, 216.16000000000008), (16.34, 308.36), (36.34, 308.36), (36.34, 216.16000000000008)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}
file_directory: sample_documents
filename: docling_paper.pdf
last_modified: 2025-12-04T13:44:43
page_number: 1
languages: ['eng']
filetype: application/pdf


In [38]:
# Common metadata fields and their uses

print("Common Metadata Fields:")
print("="*60)

metadata_info = {
    "filename": "Source file name - useful for citations",
    "file_directory": "Directory path of the source file",
    "filetype": "MIME type of the source document",
    "page_number": "Page number in the document",
    "coordinates": "Bounding box coordinates (for layout analysis)",
    "text_as_html": "HTML representation (for tables)",
    "category_depth": "Heading level (for titles)",
    "languages": "Detected languages in the text",
    "emphasized_text_contents": "Bold/italic text",
    "link_urls": "URLs found in the element",
}

for field, description in metadata_info.items():
    print(f"\n{field}:")
    print(f"  {description}")

Common Metadata Fields:

filename:
  Source file name - useful for citations

file_directory:
  Directory path of the source file

filetype:
  MIME type of the source document

page_number:
  Page number in the document

coordinates:
  Bounding box coordinates (for layout analysis)

text_as_html:
  HTML representation (for tables)

category_depth:
  Heading level (for titles)

languages:
  Detected languages in the text

emphasized_text_contents:
  Bold/italic text

link_urls:
  URLs found in the element


In [39]:
# Filter elements by metadata
# Example: Get elements from a specific page

# Get all elements from page 1
page_1_elements = [
    e for e in pdf_elements 
    if e.metadata.page_number == 1
]

print(f"Elements on page 1: {len(page_1_elements)}")
print("\nFirst 5 elements:")
for elem in page_1_elements[:5]:
    text = elem.text[:80] + "..." if len(elem.text) > 80 else elem.text
    print(f"  - {type(elem).__name__}: {text}")

Elements on page 1: 15

First 5 elements:
  - Text: 4 2 0 2 c e D 9
  - Title: ] L C . s c [
  - Text: 5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a
  - Title: Docling Technical Report
  - Title: Version 1.0


In [40]:
# Add custom metadata to elements
# Useful for tracking processing information

from datetime import datetime

# Create a copy of elements with custom metadata
processed_elements = []
for elem in html_elements[:3]:
    # Access the metadata and add custom fields
    elem.metadata.detection_origin = "unstructured-notebook-demo"
    processed_elements.append(elem)

# Verify custom metadata was added
print("Elements with custom metadata:")
for elem in processed_elements:
    print(f"  - {elem.text[:50]}...")
    print(f"    detection_origin: {elem.metadata.detection_origin}")

Elements with custom metadata:
  - 1. Overview...
    detection_origin: None
  - Document parsing is the process of analyzing and e...
    detection_origin: None
  - 1.1 Key Benefits...
    detection_origin: None


---
## OCR Configuration

Unstructured supports multiple OCR backends for extracting text from scanned documents and images.

In [41]:
# OCR options overview

ocr_options = {
    "tesseract": {
        "description": "Google's Tesseract OCR - widely used, good accuracy",
        "installation": "apt-get install tesseract-ocr",
        "languages": "100+ languages supported",
    },
    "paddle": {
        "description": "PaddleOCR - excellent for multi-language documents",
        "installation": "pip install paddlepaddle paddleocr",
        "languages": "80+ languages, strong CJK support",
    },
}

print("Available OCR Backends:")
print("="*60)

for backend, info in ocr_options.items():
    print(f"\n{backend.upper()}:")
    for key, value in info.items():
        print(f"  {key}: {value}")

Available OCR Backends:

TESSERACT:
  description: Google's Tesseract OCR - widely used, good accuracy
  installation: apt-get install tesseract-ocr
  languages: 100+ languages supported

PADDLE:
  description: PaddleOCR - excellent for multi-language documents
  installation: pip install paddlepaddle paddleocr
  languages: 80+ languages, strong CJK support


In [33]:
# OCR configuration example with Tesseract
# Note: Tesseract must be installed on your system

# Example configuration for OCR
ocr_config = {
    "strategy": "hi_res",      # Enable high-resolution processing
    "ocr_languages": "eng",    # Language(s) for OCR
    "hi_res_model_name": "yolox",  # Layout detection model
}

print("OCR Configuration Example:")
print(json.dumps(ocr_config, indent=2))

image_path = "sample_documents/vdb.png"
# Partition with OCR (if tesseract is installed)
if image_path and os.path.exists(image_path):
    try:
        ocr_elements = partition_image(
            filename=image_path,
            strategy="hi_res",
            ocr_languages="eng",
        )
        print(f"\nOCR extracted {len(ocr_elements)} elements")
        for elem in ocr_elements:
            print(f"  - {elem.text}")
    except Exception as e:
        print(f"\nOCR not available: {e}")

The ocr_languages kwarg will be deprecated in a future version of unstructured. Please use languages instead.
Only one of languages and ocr_languages should be specified. languages is preferred. ocr_languages is marked for deprecation.


OCR Configuration Example:
{
  "strategy": "hi_res",
  "ocr_languages": "eng",
  "hi_res_model_name": "yolox"
}

OCR extracted 5 elements
  - G@drant
  - QDRANT VECTOR DATABASE: TYPICAL WORKFLOW & OPERATIONS IN PRACTICE
  - SETUP INSERT DATA QUERY & REFINE MAINTAIN MONITOR & BACKUP (Foundation) (Pipeline) (Search & Filter) (Management) (Health & Safety) Sal le aN yd f=>y update_payload() ip I (External) (P=> \t ) fe Modify metadata. \ fal e7> Cn and Check collection ne wae 7 - > Update embeddings. | G J Retrieve points. a aes use filters 1 > View collection _ y leas | Remove points. COLLECTION 4 Tie mama upsert() => recommend() FILTER Find items based on positive/negative JUSS examples. with search() ee ; create_snapshot() Define collection parameters Generate vector embeddings Apply metadata filters to refine Create a point-in-time (name, dimension, metric) to and payloads from raw data, results (e.g., by category, date). backup for recovery. establish the vector space. then insert or

In [37]:
# PDF with forced OCR
# Use ocr_only strategy when you know the PDF is scanned

print("Forced OCR Configuration for Scanned PDFs:")
print("="*60)

# Example configuration
#scanned_pdf_config =
from unstructured.partition.pdf import partition_pdf

elements = partition_pdf(
    filename="sample_documents/scan.pdf",
    strategy="ocr_only",      # Force OCR processing
    ocr_languages="eng",  # Multiple languages (English + French)
)


print(f"Elements from scanned PDF: {len(elements)}")

for elem in elements:
    print(f"  - {type(elem).__name__}: {elem.text[:60]}..." if len(elem.text) > 60 else f"  - {type(elem).__name__}: {elem.text}")


The ocr_languages kwarg will be deprecated in a future version of unstructured. Please use languages instead.
Only one of languages and ocr_languages should be specified. languages is preferred. ocr_languages is marked for deprecation.


Forced OCR Configuration for Scanned PDFs:
Elements from scanned PDF: 52
  - Title: (wSins. Sinnt lousits
  - NarrativeText: i bent Pasposs ‘- Col cult the dis
  - Title: CoSing Gen’ loseity Leah low Cosive Distonce Low Hieh
  - Title: Jase st action Shi fs
  - Title: bo vectouS
  - Title: Fosemulor 7 SA, BA
  - Title: A: B vec fang
  - NarrativeText: wi multi lication, >) = Sum of slpmeal Se P
  - Title: lngth. | Magaituds of the v
  - Title: jie | > {ee
  - Title: Podon A Puckon. B
  - Title: lint] =
  - NarrativeText: h, ete ra) Roti “ “4 Romont® Rating \lictow 3
  - Title: Parson A 5 ro mp 10 & Ty mine TTY
  - Title: Step 4: Del Product A.B (sxto) + (ax)=68
  - Title: Step A lr Cole the Moqnidudis
  - NarrativeText: [Imi } = { ght Be _ fst x 583
  - Text: | (5) | = n Jo” + a =
  - Text: Apply the Josemula 68 Cosine Gina loxi ty = a3 KING x 7,
  - NarrativeText: Band BR ok resvly ‘ntical .
  - Title: \VVetose Visuel ae
  - NarrativeText: Cosa Coho A (bo > a FO A 5 —? 7 B B Cosine = 

---
## 9. LangChain Integration

Unstructured integrates seamlessly with LangChain through the `langchain-unstructured` package.

In [38]:
# Import LangChain components
from langchain_unstructured import UnstructuredLoader

print("LangChain Unstructured Integration")
print("="*60)

LangChain Unstructured Integration


In [40]:
# Basic UnstructuredLoader usage
# Load a document and convert to LangChain Document objects

# Load the sample HTML file
loader = UnstructuredLoader(
    file_path="sample_documents/docling_paper.pdf",
)

# Load documents
docs = loader.load()

print(f"Loaded {len(docs)} LangChain documents")
print()

# Show first document
if docs:
    print("First document:")
    print(f"  Content: {docs[0].page_content[:200]}...")
    print(f"  Metadata: {docs[0].metadata}")

Loaded 276 LangChain documents

First document:
  Content: 4 2 0 2 c e D 9...
  Metadata: {'source': 'sample_documents/docling_paper.pdf', 'coordinates': {'points': ((16.34, 216.16000000000008), (16.34, 308.36), (36.34, 308.36), (36.34, 216.16000000000008)), 'system': 'PixelSpace', 'layout_width': 612.0, 'layout_height': 792.0}, 'file_directory': 'sample_documents', 'filename': 'docling_paper.pdf', 'last_modified': '2025-12-04T13:44:43', 'page_number': 1, 'languages': ['eng'], 'filetype': 'application/pdf', 'category': 'UncategorizedText', 'element_id': '94656b2552b350a8a9bedcf9fe7ad9dc'}


In [41]:
# UnstructuredLoader modes
# Different modes affect how documents are split

print("UnstructuredLoader Modes:")
print("="*60)

modes = {
    "single": "Entire document as one Document object",
    "elements": "Each element as separate Document",
    "paged": "Each page as separate Document",
}

for mode, description in modes.items():
    print(f"\n{mode}:")
    print(f"  {description}")

UnstructuredLoader Modes:

single:
  Entire document as one Document object

elements:
  Each element as separate Document

paged:
  Each page as separate Document


In [45]:
# Load in "elements" mode
# Each element becomes a separate Document

loader_elements = UnstructuredLoader(
    file_path="sample_documents/docling_paper.pdf",
    mode="elements",  # Each element as separate document
)

docs_elements = loader_elements.load()

print(f"Documents in 'elements' mode: {len(docs_elements)}")
print()

print("Sample documents:")
for i, doc in enumerate(docs_elements[:5]):
    content = doc.page_content[:80] + "..." if len(doc.page_content) > 80 else doc.page_content
    print(f"  [{i}] {content}")

Documents in 'elements' mode: 276

Sample documents:
  [0] 4 2 0 2 c e D 9
  [1] ] L C . s c [
  [2] 5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a
  [3] Docling Technical Report
  [4] Version 1.0


In [50]:
# Load in "single" mode
# Entire document as one Document object

loader_single = UnstructuredLoader(
    file_path="sample_documents/docling_paper.pdf",
    mode="single",  # Entire document as one
)

docs_single = loader_single.load()

print(f"Documents in 'single' mode: {len(docs_single)}")
print(f"\nDocument length: {len(docs_single[0].page_content)} characters")
print(f"\nFirst 500 characters:")
print(docs_single[11].page_content[:1000])

Documents in 'single' mode: 276

Document length: 15 characters

First 500 characters:
Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge variability in formats, weak standardization and printing-optimized characteristic, which discards most structural features and metadata. With the advent of LLMs and popular application patterns such as retrieval-augmented generation (RAG), leveraging the rich content embedded in PDFs has become ever more relevant. In the past decade, several powerful document understanding solutions have emerged on the market, most of which are commercial soft- ware, cloud offerings [3] and most recently, multi-modal vision-language models. As of today, only a handful of open-source tools cover PDF conversion, leaving a significant feature and quality gap to proprietary solutions.


In [51]:
# UnstructuredLoader with strategy configuration
# Pass strategy and other options to control partitioning

pdf_path = "sample_documents/docling_paper.pdf"
# Load PDF with hi_res strategy
loader_pdf = UnstructuredLoader(
    file_path=pdf_path,
    mode="elements",
    strategy="fast",  # Use fast for this demo
)

docs_pdf = loader_pdf.load()

print(f"PDF documents loaded: {len(docs_pdf)}")
print()

# Show element types in metadata
print("Document metadata examples:")
for doc in docs_pdf[:3]:
    print(f"  Category: {doc.metadata.get('category', 'N/A')}")
    print(f"  Page: {doc.metadata.get('page_number', 'N/A')}")
    print(f"  Content: {doc.page_content[:60]}...")
    print()

PDF documents loaded: 276

Document metadata examples:
  Category: UncategorizedText
  Page: 1
  Content: 4 2 0 2 c e D 9...

  Category: Title
  Page: 1
  Content: ] L C . s c [...

  Category: UncategorizedText
  Page: 1
  Content: 5 v 9 6 8 9 0 . 8 0 4 2 : v i X r a...



In [52]:
# Load multiple files with UnstructuredLoader
# You can load multiple documents at once
import os 
# List of files to load
files_to_load = [
    "sample_documents/sample.html",
    "sample_documents/sample.md",
]

# Load all files
all_docs = []
for file_path in files_to_load:
    if os.path.exists(file_path):
        loader = UnstructuredLoader(
            file_path=file_path,
            mode="elements",
        )
        docs = loader.load()
        all_docs.extend(docs)
        print(f"Loaded {len(docs)} documents from {file_path}")

print(f"\nTotal documents: {len(all_docs)}")

Loaded 20 documents from sample_documents/sample.html
Loaded 44 documents from sample_documents/sample.md

Total documents: 64


---
## 10. Complete RAG Application Example

Let's build a complete RAG (Retrieval-Augmented Generation) application using:
- Unstructured for document parsing
- LangChain for the pipeline
- ChromaDB for vector storage
- OpenAI for embeddings and LLM

In [53]:
!uv pip show langchain

Name: langchain
Version: 0.3.27
Location: /Users/sourangshupal/Downloads/document-parsers-rag/.venv/lib/python3.12/site-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, pyyaml, requests, sqlalchemy
Required-by: langchain-community


In [54]:
# RAG Application Setup
# Import all required components

from langchain_unstructured import UnstructuredLoader
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_classic.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

# Verify API key is available
if not os.getenv("OPENAI_API_KEY"):
    print("WARNING: OPENAI_API_KEY not set. RAG example will not work.")
    print("Set it in your .env file or environment variables.")
else:
    print("OpenAI API key found. Ready for RAG example.")

OpenAI API key found. Ready for RAG example.


In [55]:
# Step 1: Load documents using UnstructuredLoader
# We'll use multiple document types to demonstrate flexibility

def load_documents(file_paths):
    """
    Load multiple documents using UnstructuredLoader.
    
    Args:
        file_paths: List of file paths to load
    
    Returns:
        List of LangChain Document objects
    """
    all_docs = []
    
    for file_path in file_paths:
        if os.path.exists(file_path):
            print(f"Loading: {file_path}")
            
            loader = UnstructuredLoader(
                file_path=file_path,
                mode="elements",  # Get individual elements
                strategy="fast",  # Fast processing
            )
            
            docs = loader.load()
            all_docs.extend(docs)
            print(f"  -> Loaded {len(docs)} elements")
        else:
            print(f"File not found: {file_path}")
    
    return all_docs

# Load our sample documents
document_paths = [
    "sample_documents/sample.html",
    "sample_documents/sample.md",
    "sample_documents/sample.pdf",
    "sample_documents/sample.docx",
    "sample_documents/sample.xlsx",
    "sample_documents/sample.pptx",
]

documents = load_documents(document_paths)
print(f"\nTotal documents loaded: {len(documents)}")

Loading: sample_documents/sample.html
  -> Loaded 20 elements
Loading: sample_documents/sample.md
  -> Loaded 44 elements
File not found: sample_documents/sample.pdf
Loading: sample_documents/sample.docx
  -> Loaded 53 elements
Loading: sample_documents/sample.xlsx
  -> Loaded 11 elements
Loading: sample_documents/sample.pptx
  -> Loaded 23 elements

Total documents loaded: 151


In [57]:
# Step 2: Split documents into chunks
# While UnstructuredLoader can chunk, we'll use LangChain's splitter for more control

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # Maximum chunk size
    chunk_overlap=50,      # Overlap between chunks
    length_function=len,
    separators=["\n\n", "\n", " ", ""],
)

# Split the documents
split_docs = text_splitter.split_documents(documents)

print(f"Documents after splitting: {len(split_docs)}")
print(f"\nSample chunk:")
print(f"  Content: {split_docs[1].page_content[:400]}...")
print(f"  Metadata: {split_docs[1].metadata}")

Documents after splitting: 155

Sample chunk:
  Content: Document parsing is the process of analyzing and extracting structured information from various document formats. This includes PDFs, Word documents, HTML pages, and more....
  Metadata: {'source': 'sample_documents/sample.html', 'last_modified': '2025-11-28T09:19:12', 'languages': ['eng'], 'file_directory': 'sample_documents', 'filename': 'sample.html', 'filetype': 'text/html', 'parent_id': '2a000fb58dc445e313c5dffe0ef3a614', 'category': 'NarrativeText', 'element_id': '0111c3838e668907531ba2247a8d97b1'}


In [59]:
# Step 3: Create embeddings and vector store
# Using OpenAI embeddings and Chroma for storage
from langchain_community.vectorstores.utils import filter_complex_metadata


if os.getenv("OPENAI_API_KEY"):
    # Initialize OpenAI embeddings
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",  # Cost-effective embedding model
    )
    
    # Create Chroma vector store
    # persist_directory saves the database locally for reuse
    split_docs = filter_complex_metadata(split_docs)
        #documents=split_docs,
        #metadata_keys=["source", "page", "chunk"],
    
    vectorstore = Chroma.from_documents(
        documents=split_docs,
        embedding=embeddings,
        persist_directory="./chroma_db_unstructured",
        collection_name="unstructured_rag_demo",
    )
    
    print(f"Vector store created with {len(split_docs)} documents")
    print(f"Persisted to: ./chroma_db_unstructured")
else:
    print("Skipping vector store creation - no API key")
    vectorstore = None

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Vector store created with 155 documents
Persisted to: ./chroma_db_unstructured


In [60]:
# Step 4: Test similarity search
# Verify the vector store is working correctly

if vectorstore:
    # Search for relevant documents
    query = "What is document parsing?"
    
    results = vectorstore.similarity_search(
        query=query,
        k=3,  # Return top 3 results
    )
    
    print(f"Query: {query}")
    print("\nTop 3 relevant chunks:")
    print("="*60)
    
    for i, doc in enumerate(results):
        print(f"\nResult {i+1}:")
        print(f"  Source: {doc.metadata.get('filename', 'Unknown')}")
        print(f"  Content: {doc.page_content[:400]}...")

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Query: What is document parsing?

Top 3 relevant chunks:

Result 1:
  Source: sample.html
  Content: Document parsing is the process of analyzing and extracting structured information from various document formats. This includes PDFs, Word documents, HTML pages, and more....

Result 2:
  Source: sample.html
  Content: Here's a simple example of using a document parser:...

Result 3:
  Source: sample.md
  Content: Document Parsing Best Practices...


In [61]:
# Step 5: Create the RAG chain
# Combine retrieval with LLM for question answering

if vectorstore and os.getenv("OPENAI_API_KEY"):
    # Initialize the LLM
    llm = ChatOpenAI(
        model="gpt-4o-mini",  # Cost-effective model
        temperature=0,         # Deterministic responses
    )
    
    # Create the retriever
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5},  # Retrieve top 5 chunks
    )
    
    # Create the RAG chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Stuff all context into prompt
        retriever=retriever,
        return_source_documents=True,  # Include sources in response
    )
    
    print("RAG chain created successfully!")
else:
    print("Cannot create RAG chain - missing dependencies")
    qa_chain = None

RAG chain created successfully!


In [62]:
# Step 6: Query the RAG system
# Ask questions about the documents

def ask_question(qa_chain, question):
    """
    Ask a question using the RAG chain.
    
    Args:
        qa_chain: The RetrievalQA chain
        question: Question to ask
    
    Returns:
        Answer and source documents
    """
    print(f"Question: {question}")
    print("="*60)
    
    result = qa_chain.invoke({"query": question})
    
    print(f"\nAnswer: {result['result']}")
    
    print(f"\nSources:")
    for i, doc in enumerate(result['source_documents']):
        source = doc.metadata.get('filename', 'Unknown')
        print(f"  {i+1}. {source}")
    
    return result

# Ask questions
if qa_chain:
    # Question 1
    result1 = ask_question(
        qa_chain, 
        "What are the key benefits of document parsing?"
    )
    
    print("\n" + "="*60 + "\n")
    
    # Question 2
    result2 = ask_question(
        qa_chain,
        "What file formats are supported for document parsing?"
    )

Question: What are the key benefits of document parsing?


INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Answer: The key benefits of document parsing include:

1. **Structured Information Extraction**: It allows for the extraction of structured data from unstructured documents, making it easier to analyze and utilize the information.

2. **Improved Data Accessibility**: By converting unstructured data into a structured format, it enhances data accessibility and usability for various applications.

3. **Automation of Data Processing**: Document parsing automates the process of data entry and processing, reducing manual effort and the potential for human error.

4. **Enhanced Decision-Making**: With structured data readily available, organizations can make more informed decisions based on accurate and timely information.

5. **Integration with AI Applications**: Document parsing is a critical component in modern AI applications, enabling better data handling and analysis.

6. **Support for Various Formats**: It can handle multiple document formats, such as PDFs, Word documents, and HTML pa

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Answer: Document parsing supports various file formats, including PDFs, Word documents, HTML pages, and more.

Sources:
  1. sample.html
  2. sample.html
  3. sample.html
  4. sample.md
  5. sample.html


In [63]:
# Step 7: Interactive RAG Demo
# A reusable function for asking questions

def rag_query(question):
    """
    Simplified function to query the RAG system.
    
    Args:
        question: Your question about the documents
    
    Returns:
        The answer string
    """
    if not qa_chain:
        return "RAG chain not available. Please set OPENAI_API_KEY."
    
    result = qa_chain.invoke({"query": question})
    return result['result']

# Example usage
if qa_chain:
    answer = rag_query("What are the applications of document parsing in RAG systems?")
    print("Answer:", answer)

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Answer: I don't know.


### Error Handling

In [64]:
# Robust error handling for document processing

def safe_partition(file_path, **kwargs):
    """
    Safely partition a document with error handling.
    
    Args:
        file_path: Path to the document
        **kwargs: Additional arguments for partition()
    
    Returns:
        Tuple of (elements, error_message)
    """
    try:
        # Check if file exists
        if not os.path.exists(file_path):
            return None, f"File not found: {file_path}"
        
        # Check file size (warn for large files)
        file_size = os.path.getsize(file_path)
        if file_size > 100 * 1024 * 1024:  # 100 MB
            print(f"Warning: Large file ({file_size / 1024 / 1024:.1f} MB)")
        
        # Partition the document
        elements = partition(filename=file_path, **kwargs)
        
        return elements, None
        
    except FileNotFoundError:
        return None, "File not found"
    except PermissionError:
        return None, "Permission denied"
    except Exception as e:
        return None, f"Unexpected error: {str(e)}"

# Test with various scenarios
test_files = [
    "sample_documents/sample.html",
    "nonexistent_file.pdf",
    "sample_documents/sample.md",
]

print("Safe Partition Results:")
print("="*60)

for file_path in test_files:
    elements, error = safe_partition(file_path)
    
    if error:
        print(f"\n{file_path}: ERROR - {error}")
    else:
        print(f"\n{file_path}: SUCCESS - {len(elements)} elements")

Safe Partition Results:

sample_documents/sample.html: SUCCESS - 20 elements

nonexistent_file.pdf: ERROR - File not found: nonexistent_file.pdf

sample_documents/sample.md: SUCCESS - 44 elements


---
## 12. Summary & Best Practices

### Key Takeaways

1. **Use the right strategy**:
   - `fast` for quick processing of simple documents
   - `hi_res` for complex layouts and accurate table extraction
   - `ocr_only` for scanned documents

2. **Choose appropriate chunking**:
   - `chunk_by_title` preserves document structure
   - Use overlap for better context in RAG
   - Match chunk size to your embedding model's limits

3. **Leverage metadata**:
   - Page numbers for citations
   - Element types for filtering
   - File information for source tracking

4. **Error handling**:
   - Always wrap partition calls in try-except
   - Check file existence and permissions
   - Log errors for debugging

### Recommended Workflow

```python
# 1. Load documents
elements = partition(filename="document.pdf", strategy="fast")

# 2. Filter relevant elements
elements = [e for e in elements if len(e.text) > 10]

# 3. Chunk for RAG
chunks = chunk_by_title(elements, max_characters=1000)

# 4. Create embeddings and store
vectorstore = Chroma.from_documents(chunks, embeddings)

# 5. Query with RAG
qa_chain = RetrievalQA.from_chain_type(llm, retriever=vectorstore.as_retriever())
result = qa_chain.invoke({"query": "Your question"})
```

In [ ]:
# Cleanup: Remove temporary files and vector store

import shutil

cleanup_paths = [
    "./chroma_db_unstructured",
    "sample_documents/elements_export.json",
]

print("Cleanup (optional):")
for path in cleanup_paths:
    if os.path.exists(path):
        print(f"  Would remove: {path}")

# Uncomment to actually clean up:
# for path in cleanup_paths:
#     if os.path.isdir(path):
#         shutil.rmtree(path)
#     elif os.path.isfile(path):
#         os.remove(path)
# print("Cleanup complete!")

---
## Resources

- [Unstructured Documentation](https://docs.unstructured.io/)
- [Unstructured GitHub](https://github.com/Unstructured-IO/unstructured)
- [LangChain Unstructured Integration](https://python.langchain.com/docs/integrations/document_loaders/unstructured_file)
- [Supported File Types](https://docs.unstructured.io/open-source/introduction/supported-file-types)

---

*This notebook was created as a comprehensive guide to document parsing with Unstructured for RAG systems.*